<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [3]:
import folium
import pandas as pd

In [4]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [6]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [7]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [8]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [9]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [10]:
# Initialize the map centered at NASA Johnson Space Center
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add a circle and marker for each launch site
for index, row in launch_sites_df.iterrows():
    # Extract latitude, longitude, and site name
    lat = row['Lat']
    lon = row['Long']
    site_name = row['Launch Site']
    
    # Add a circle at the launch site
    folium.Circle(
        location=[lat, lon],
        radius=1000,
        color='#007849',
        fill=True,
        fill_color='#31a354',
        fill_opacity=0.6,
        popup=folium.Popup(site_name, parse_html=True)
    ).add_to(site_map)
    
    # Add a marker with the site name
    folium.Marker(
        location=[lat, lon],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12px; color:#007849;"><b>%s</b></div>' % site_name,
        )
    ).add_to(site_map)

# Display the map
site_map



The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [11]:
# Task 2: Mark the success/failed launches for each site on the map


# Load the SpaceX launch site data
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df = pd.read_csv(spacex_csv_file)

# Initialize a Folium map centered at NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Iterate through each launch record in spacex_df
for index, row in spacex_df.iterrows():
    # Extract latitude, longitude, site name, and success class
    lat = row['Lat']
    lon = row['Long']
    site_name = row['Launch Site']
    launch_class = row['class']  # 1 for success, 0 for failure
    
    # Define circle color based on launch success or failure
    circle_color = 'green' if launch_class == 1 else 'red'
    
    # Add a circle marker for each launch
    folium.Circle(
        location=[lat, lon],
        radius=100,
        color=circle_color,
        fill=True,
        fill_color=circle_color,
        fill_opacity=0.6,
        popup=folium.Popup(f"{site_name} - {'Success' if launch_class == 1 else 'Failure'}", parse_html=True)
    ).add_to(site_map)

# Display the map
site_map



Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [12]:
spacex_df.tail(10)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
46,43,2017-10-11,22:53:00,F9 FT B1031.2,KSC LC-39A,SES-11 / EchoStar 105,5200.00,GTO,SES EchoStar,Success (drone ship),1,28.573255,-80.646895
47,44,2017-10-30,19:34:00,F9 B4 B1042.1,KSC LC-39A,Koreasat 5A,3500.00,GTO,KT Corporation,Success (drone ship),1,28.573255,-80.646895
48,54,2018-05-11,20:14:00,F9 B5 B1046.1,KSC LC-39A,Bangabandhu-1,3600.00,GTO,Thales-Alenia/BTRC,Success (drone ship),1,28.573255,-80.646895
49,45,2017-12-15,15:36:00,F9 FT B1035.2,CCAFS SLC-40,SpaceX CRS-13,2205.00,LEO (ISS),NASA (CRS),Success (ground pad),1,28.563197,-80.576820
50,47,2018-01-08,1:00:00,F9 B4 B1043.1,CCAFS SLC-40,Zuma,3696.65,LEO,Northrop Grumman,Success (ground pad),1,28.563197,-80.576820
51,48,2018-01-31,21:25:00,F9 FT B1032.2,CCAFS SLC-40,GovSat-1 / SES-16,4230.00,GTO,SES,Controlled (ocean),0,28.563197,-80.576820
52,50,2018-03-06,5:33:00,F9 B4 B1044,CCAFS SLC-40,Hispasat 30W-6 PODSat,6092.00,GTO,Hispasat NovaWurks,No attempt,0,28.563197,-80.576820
53,52,2018-04-02,20:30:00,F9 B4 B1039.2,CCAFS SLC-40,SpaceX CRS-14,2647.00,LEO (ISS),NASA (CRS),No attempt,0,28.563197,-80.576820
54,53,2018-04-18,22:51:00,F9 B4 B1045.1,CCAFS SLC-40,Transiting Exoplanet Survey Satellite (TESS),362.00,HEO,NASA (LSP),Success (drone ship),1,28.563197,-80.576820
55,56,2018-06-04,4:45:00,F9 B4 B1040.2,CCAFS SLC-40,SES-12,5384.00,GTO,SES,No attempt,0,28.563197,-80.576820


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [13]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [14]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

# Create a new column 'marker_color' based on the 'class' column
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Verify the new column
print(spacex_df[['class', 'marker_color']].head())

# Initialize a Folium map centered at NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)

# Create a MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# Add markers to the MarkerCluster
for index, row in spacex_df.iterrows():
    # Extract latitude, longitude, site name, and marker color
    lat = row['Lat']
    lon = row['Long']
    site_name = row['Launch Site']
    marker_color = row['marker_color']

    # Create a Marker and add it to the MarkerCluster
    folium.Marker(
        location=[lat, lon],
        icon=folium.Icon(color=marker_color),
        popup=folium.Popup(f"{site_name} - {'Success' if row['class'] == 1 else 'Failure'}", parse_html=True)  # Added popup
    ).add_to(marker_cluster)

# Display the map
site_map

   class marker_color
0      0          red
1      0          red
2      0          red
3      0          red
4      0          red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [15]:
# Initialize a Folium map centered at NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Create a MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# Iterate through each row in spacex_df to add markers to the cluster
for index, record in spacex_df.iterrows():
    # Extract latitude, longitude, site name, and success/failure class
    lat = record['Lat']
    lon = record['Long']
    site_name = record['Launch Site']
    launch_class = record['class']  # 1 for success, 0 for failure
    
    # Define marker color based on class
    marker_color = 'green' if launch_class == 1 else 'red'
    
    # Create a Marker object with customized icon and add to marker_cluster
    marker = folium.Marker(
        location=[lat, lon],
        icon=folium.Icon(color='white', icon_color=marker_color),
        popup=folium.Popup(f"{site_name} - {'Success' if launch_class == 1 else 'Failure'}", parse_html=True)
    )
    marker_cluster.add_child(marker)

# Add the marker cluster to the map
site_map.add_child(marker_cluster)

# Display the map
site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [17]:
# TASK 3: Calculate the distances between a launch site to its proximities
import math


# Example DataFrame for proximities (e.g., cities, landmarks)
proximities = pd.DataFrame({
    'Name': ['Proximity1', 'Proximity2', 'Proximity3'],
    'Lat': [28.5634, 28.4418, 28.5721],  # Example latitudes
    'Long': [-80.5768, -80.5811, -80.6480]  # Example longitudes
})

# Example coordinates of the launch site (e.g., CCAFS SLC-40)
launch_site_coords = (28.5623, -80.5774)

# Function to calculate Euclidean distance
def calculate_distance(lat1, lon1, lat2, lon2):
    return math.sqrt((lat2 - lat1)**2 + (lon2 - lon1)**2)

# Calculate distances and add them as a new column in the DataFrame
proximities['Distance (degrees)'] = proximities.apply(
    lambda row: calculate_distance(launch_site_coords[0], launch_site_coords[1], row['Lat'], row['Long']),
    axis=1
)

# Display the updated DataFrame with distances
print(proximities)


         Name      Lat     Long  Distance (degrees)
0  Proximity1  28.5634 -80.5768            0.001253
1  Proximity2  28.4418 -80.5811            0.120557
2  Proximity3  28.5721 -80.6480            0.071277


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [18]:
import folium
from folium.plugins import MousePosition

# Initialize a Folium map centered at NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add Mouse Position plugin to display coordinates on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',  # Display position of the coordinates on the map
    separator=' Long: ',  # Separator between latitude and longitude values
    empty_string='NaN',  # Value to display when no coordinates are available
    lng_first=False,  # Display latitude first (False) or longitude first (True)
    num_digits=20,  # Number of decimal places for coordinates
    prefix='Lat:',  # Prefix for the coordinate display
    lat_formatter=formatter,  # Formatter for latitude values
    lng_formatter=formatter   # Formatter for longitude values
)

# Add the MousePosition plugin to the map
site_map.add_child(mouse_position)

# Display the map
site_map


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [19]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [21]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Coordinates of a SpaceX launch site (e.g., CCAFS SLC-40)
launch_site_lat = 28.5623
launch_site_lon = -80.5774

# Initialize a Folium map centered at the launch site
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

# Add MousePosition plugin to get coordinates dynamically
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)

# Manually identified closest coastline point (using MousePosition)
coastline_lat = 28.56367  # Example latitude from MousePosition
coastline_lon = -80.57163  # Example longitude from MousePosition

# Calculate distance between launch site and coastline
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(f"Distance to closest coastline: {distance_coastline:.2f} km")

# Add markers for both points on the map
folium.Marker(
    location=[launch_site_lat, launch_site_lon],
    popup="Launch Site",
    icon=folium.Icon(color='blue', icon='info-sign')
).add_to(site_map)

folium.Marker(
    location=[coastline_lat, coastline_lon],
    popup="Closest Coastline",
    icon=folium.Icon(color='green', icon='info-sign')
).add_to(site_map)

# Display the map
site_map


Distance to closest coastline: 0.58 km


In [22]:
# Coordinates of a SpaceX launch site (e.g., CCAFS SLC-40)
launch_site_lat = 28.5623
launch_site_lon = -80.5774

# Coordinates of the closest coastline point (manually identified using MousePosition)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate the distance between the launch site and the coastline
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Initialize a Folium map centered at the launch site
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=15)

# Add a marker for the launch site
folium.Marker(
    location=[launch_site_lat, launch_site_lon],
    popup="Launch Site",
    icon=folium.Icon(color='blue', icon='info-sign')
).add_to(site_map)

# Add a marker for the closest coastline point with distance displayed as an icon property
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_coastline),
    ),
    popup="Closest Coastline"
)
site_map.add_child(distance_marker)

# Display the map
site_map


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [23]:
# Coordinates of a SpaceX launch site (e.g., CCAFS SLC-40)
launch_site_lat = 28.5623
launch_site_lon = -80.5774

# Coordinates of the closest coastline point (manually identified using MousePosition)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate the distance between the launch site and the coastline
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Initialize a Folium map centered at the launch site
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=15)

# Add a marker for the launch site
folium.Marker(
    location=[launch_site_lat, launch_site_lon],
    popup="Launch Site",
    icon=folium.Icon(color='blue', icon='info-sign')
).add_to(site_map)

# Add a marker for the closest coastline point with distance displayed as an icon property
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_coastline),
    ),
    popup="Closest Coastline"
)
site_map.add_child(distance_marker)

# Create a PolyLine connecting the launch site and the closest coastline point
coordinates = [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]]
lines = folium.PolyLine(locations=coordinates, weight=2, color='blue')
site_map.add_child(lines)

# Display the map
site_map



Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [24]:
# Coordinates of a SpaceX launch site (e.g., CCAFS SLC-40)
launch_site_lat = 28.5623
launch_site_lon = -80.5774

# Coordinates of the closest city/railway/highway (manually identified or pre-defined)
closest_location_lat = 28.6100  # Example latitude for closest city/railway/highway
closest_location_lon = -80.8070  # Example longitude for closest city/railway/highway

# Calculate the distance between the launch site and the closest location
distance_to_closest_location = calculate_distance(
    launch_site_lat, launch_site_lon, closest_location_lat, closest_location_lon
)

# Initialize a Folium map centered at the launch site
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

# Add a marker for the launch site
folium.Marker(
    location=[launch_site_lat, launch_site_lon],
    popup="Launch Site",
    icon=folium.Icon(color='blue', icon='info-sign')
).add_to(site_map)

# Add a marker for the closest location with distance displayed as an icon property
distance_marker = folium.Marker(
    location=[closest_location_lat, closest_location_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_to_closest_location),
    ),
    popup="Closest Location"
)
site_map.add_child(distance_marker)

# Create a PolyLine connecting the launch site and the closest location
coordinates = [[launch_site_lat, launch_site_lon], [closest_location_lat, closest_location_lon]]
lines = folium.PolyLine(locations=coordinates, weight=2, color='green')
site_map.add_child(lines)

# Display the map
site_map



After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
